<a href="https://colab.research.google.com/github/AnnSenina/Python_CL_2025/blob/main/notebooks/Python_7_NLTK%2C_%D1%87%D0%B0%D1%81%D1%82%D0%BE%D1%82%D0%BD%D1%8B%D0%B5_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B8%2C_n_%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLTK, частотные списки, n-граммы

Подготовка текста для анализа

<s>Ведь мы c вами все знаем, что на самом деле цифровой анализ текста - это и есть частотности слов</s> :)

Используются материалы из тетрадок Д.Скоринкина, А. Хорошевой

In [ ]:
# кое-что мы уже умеем, например:
text = " Давайте нормализуем этот текст!      "

print(text.lower().strip(" !").split())

In [ ]:
# или так:
import re
print(re.sub(r'\W', ' ', text.lower()).split())

Но пока мы получаем скорее псевдо-токены, например, слова с дефисом грубо делятся пополам

В идеале нужно использовать какую-нибудь умную библиотеку, которая сама будет определять, нужно ли разрезать слово пополам или же оставить, как есть

Классический вариант - nltk (хотя нарезать на токены будут уметь и другие библиотеки - mystem, spacy и т.д.)

### Попробуем на реальном тексте
По инерции от моих коллег берем в наследство пример с "Преступлением и наказанием"

Проведем предобработку текста. Посмотрим на практике, на каком этапе нужна лемматизация.

In [ ]:
with open('Dostoevsky_PrestuplenieINakazanie.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
# нулевой шаг - переводим текст в нижний регистр
text = text.lower()

Убедимся, что в тексте лежит то что мы ожидаем:

In [ ]:
# начало романа:
print(text[:100])

In [ ]:
# посмотрим на все слова, в которых есть подстрока 'топор'
for token in text.split():
  if 'топор' in token:
    print(token)

In [ ]:
# чистим регуляркой
# проверим, что проблема приклеившейся пунктуации ушла
for word in re.sub(r'\W', ' ', text.lower()).split():
  if 'топор' in word:
    print(word)
# в принципе, работает!

### Еще более умный способ:

сегментируем текст готовым токенизатором — возьмем его из прекрасной библиотеки для обработки языка NLTK

[Документация по NLTK](https://www.nltk.org/) и [книжка](https://www.nltk.org/book/)

In [ ]:
!pip install nltk
# %pip install nltk # в VS Code нужно устанавливать так

# в Colab уже есть, в других средах - установите
# в PyCharm мы можем устанавливать библиотеки в терминале (без знака !)

In [ ]:
# библиотеки и импорты
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize

# это подгрузка вспомогательных данных, которые нужны nltk для токенизации
from nltk import download
# download('punkt')
# 'punkt' стал неактуальным в новой версии nltk (устаревший и небезопасный)
# в новых версиях используется 'punkt_tab'
# если у вас не работает с punct_tab - значит, версия nltk более старая (не страшно), раскоментируйте import punkt
download('punkt_tab')

# стоп-слова
download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

# стемминг
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

# разделение на предложения
from nltk.tokenize import sent_tokenize

Сначала разбираемся с токенизатором на текстике попроще и покороче

In [ ]:
text_2 = """Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE. Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.
Во-первых, NLI можно использовать для контроля качества генеративных моделей. Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод. Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя". С помощью модели NLI можно проверять, что из X следует Y (то есть в новом тексте нету "отсебятины", придуманной моделью), и что из Y следует X (т.е. вся информация, присутствовавшая в исходном тексте, в новом также отражена).
Во-вторых, с помощью моделей NLI можно находить нетривиальные парафразы и в целом определять смысловую близость текстов. Для русского языка уже существует ряд моделей и датасетов по перефразированию, но кажется, что можно сделать ещё больше и лучше. В статье Improving Paraphrase Detection with the Adversarial Paraphrasing Task предложили считать парафразами такую пару предложений, в которой каждое логически следует из другого – и это весьма логично. Поэтому модели NLI можно использовать и для сбора обучающего корпуса парафраз (и не-парафраз, если стоит задача их детекции), и для фильтрации моделей, генерирующих парафразы."""
# текст отсюда - https://habr.com/ru/post/582620/

In [ ]:
print(wordpunct_tokenize(text_2))
# wordpunct_tokenizer разбирает по регулярке - '\w+|[^\w\s]+'
# word_tokenize - тоже основан на регулярках, но более умных (учитывается последовательность некоторых символов, символы начала, конца слова и т.д).
# для русского языка работает немного хуже, чем для английского

In [ ]:
print(word_tokenize(text_2))

In [ ]:
# давайте быстро найдем различия:
print(set(wordpunct_tokenize(text_2)) - set(word_tokenize(text_2)))
print(set(word_tokenize(text_2)) - set(wordpunct_tokenize(text_2)))

In [ ]:
print(len(wordpunct_tokenize(text_2)))

In [ ]:
print(len(word_tokenize(text_2)))

In [ ]:
# вернемся к Достоевскому, используем более умную токенизацию nltk
text_list_nltk = word_tokenize(text)
print(text_list_nltk[:30])

### Counter - встроенный счетчик частотностей в Python

Мы его уже видели, в домашнем задании у вас ручной сбор словаря частот, а Counter быстро собирает этот словарь сам

По сути, Counter заменяет ручную сборку словаря частотностей:

```
word_freqs = {}
for i in text_list_nltk:
  if i not in word_freqs:
    word_freqs[i] = 1
  else:
    word_freqs[i] += 1
    
word_freqs = Counter(text_list_nltk) # то же самое
```



In [ ]:
from collections import Counter

In [ ]:
# Counter - это специальный объект, который умеет легко считать повторяющиеся элементы в iterable
# Например в строке:
char_freqs = Counter('aaaaabbbbcccddefghik') # получим частотности всех элементов строки
print(char_freqs)
print(char_freqs.most_common(3)) # и топ-3 (5, 10 и т.д...) самых частотных элемента

In [ ]:
word_freqs = Counter(text_list_nltk) # отправим в Counter
word_freqs.most_common(10) # получим топ 10 "слов"

nltk сохраняет пунктуацию как отдельный токен - избавимся от нее

In [ ]:
# вспомним старые способы

print(','.isalnum()) # True, если перед нами буква или цифра, НЕ знак препинания, НЕ пробел
print('Санкт-Петербург'.isalnum()) # из-за дефиса тоже будет False

# но можно проверять этим методом, что 'Санкт-Петербург' все-таки токен, а не просто знак пунктуации
print('Санкт-Петербург'[0].isalnum())

# 'Санкт-Петербург' -> False из-за дефиса
# 'Санкт-Петербург'[0] -> 'С' -> True, т.к. начинается с буквы
# ',' -> False
# теперь все сработает

In [ ]:
text_without_punkt = []
for word in text_list_nltk:
    if word[0].isalnum():
        text_without_punkt.append(word)

# на практике вы часто встретите списковые включения:
# text_without_punkt = [word for word in text_list_nltk if word[0].isalnum()]
# краткая форма записи цикла for

In [ ]:
print(text_without_punkt[:30])

In [ ]:
Counter(text_without_punkt).most_common(30)

### Чистим от стоп-слов

Можно загрузить свои стоп-слова и удалить их, но проще взять из NLTK: там есть наборы стоп-слов для разных языков


In [ ]:
# download('stopwords')
# from nltk.corpus import stopwords
print(stopwords.fileids()) # доступные языки

In [ ]:
stop_words = stopwords.words('russian')
print(stop_words)

In [ ]:
text_clean = []
for word in text_without_punkt:
  if word not in stop_words:
    text_clean.append(word)

# text_clean = [word for word in text_without_punkt if word not in stop_words]

In [ ]:
print(text_clean[:200])

In [ ]:
Counter(text_clean).most_common(30)
# хочется, конечно, удалить побольше стоп-слов
# можно поискать другие библиотеки дополнительно или просто списки стоп-слов в интеренете
# в рамках проектов не стесняйтесь чистить больше

# кроме того, stop_words - список
# вы можете его просто дополнить:
# stop_words = stop_words + ['это', 'всё', 'очень', 'то', 'что', 'с'] # и т.д.

###  Стемминг в Python

Самый простой способ автоматический нормализации слов в языках с морфологией — стемминг. Стемминг — это очень грубое разбиение формы на предполагаемую основу и предполагаемую флексию.

Программы-стеммеры умеют превращать:
* "Vyshka's students coded" в "Vyshka student code"
* 'Маша поехала за грибами' в 'Маш поехал за гриб'
* 'Даня работает в Вышке' в "Дан работа в Вышк"

Как можно догадаться из этих примеров, стемминг — не лучшее (и крайне непопулярное) решение для языков типа русского. Он лучше подходит для английского.

В NLTK есть готовая реализация стеммера для русского языка. Давайте потестируем ее

Самый известный стеммер - стеммер Портера (или snowball стеммер).

Подробнее про стеммер Портера можно почитать [вот тут](https://medium.com/@eigenein/стеммер-портера-для-русского-языка-d41c38b2d340)

А совсем подробнее [вот тут](http://snowball.tartarus.org/algorithms/russian/stemmer.html)

In [ ]:
# from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian") # в эту переменную мы сохраним уже готовый объект-стеммер для русского

In [ ]:
# предположите, что выдаст стеммер?
print(stemmer.stem('университетами'))
print(stemmer.stem('мышам'))
print(stemmer.stem('конями'))
print(stemmer.stem('людей'))

In [ ]:
text_stemmed = []
for word in text_clean:
  text_stemmed.append(stemmer.stem(word))

# аналогично text_stemmed = [stemmer.stem(word) for word in text_clean]
print(text_stemmed[:20])

In [ ]:
Counter(text_stemmed).most_common(50)
# этот вариант должен нас убедить, что без лемматизации все плохо :)

### Сегментация на предложения

NLTK также умеет разбивать текст на предложения

In [ ]:
# from nltk.tokenize import sent_tokenize

sent_tokenize(text)[:20]

### Задание



Соберем функцию, которая берет текст (строку), приводит к нижнему регистру, токенизирует, удаляет пунктуацию и стоп-слова (давайте не включать стемминг)

In [ ]:
def clean_tokens(text):
  # ваш код ниже


  return text_clean

In [ ]:
# не забудьте испытать функцию на Достоевском

Counter(clean_tokens(text)).most_common(50)

Решение ниже

In [ ]:
# @title
def clean_tokens(text):
  text = text.lower()
  text_list_nltk = word_tokenize(text)
  text_without_punkt = [word for word in text_list_nltk if word[0].isalnum()]
  text_clean = [word for word in text_without_punkt if word not in stop_words]
  return text_clean

Для большинства европейских языков вы найдете готовые наборы стоп-слов, пунктуация будет удаляться аналогично

Но в китайском точно свои проблемы :)

Давайте сегодня без токенизации китайского, бирманского и других подобных языков

# Добавим n-граммы и визуализации

In [ ]:
# еще импорты

from nltk import Text as nltk_text

# облако слов
!pip install wordcloud # %pip install wordcloud # для VS Code
from wordcloud import WordCloud
import matplotlib.pyplot as plt

!pip install scikit-learn # %pip install scikit-learn # для VS Code
from sklearn.feature_extraction.text import TfidfVectorizer

Документация облака слов [здесь](https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html)

In [ ]:
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

# Генерируем облако слов
wordcloud = WordCloud().generate_from_frequencies(Counter(clean_tokens(text)))
plt.imshow(wordcloud) # Что изображаем
plt.axis("off") # без подписей на осях
plt.show() # показать изображение

In [ ]:
# давайте отрисуем только слова, которые встречаются хотя бы 10 раз
# добавляем аргумент margin

wordcloud = WordCloud(margin = 10).generate_from_frequencies(Counter(clean_tokens(text)))
plt.imshow(wordcloud) # Что изображаем
plt.axis("off") # без подписей на осях
plt.show() # показать изображение

In [ ]:
wordcloud = WordCloud(width = 2000,
                      height = 1500,
                      background_color='white',
                      margin = 10,
                      colormap='viridis').generate_from_frequencies(Counter(clean_tokens(text)))
plt.figure(figsize=(20, 15)) # Устанавливаем размер картинки
plt.imshow(wordcloud) # Что изображаем
plt.axis("off") # Без подписей на осях
plt.show() # показать изображение


Больше цветовых схем [здесь](https://matplotlib.org/stable/users/explain/colors/colormaps.html)

In [ ]:
# еще один способ визуализации частот встроен в NLTK
# from nltk import Text as nltk_text
nltk_format = nltk_text(clean_tokens(text))
nltk_format.dispersion_plot(['раскольников', 'соня', 'разумихин',
                             'свидригайлов', 'дуня']);

### N-gramms
Текст можно разделить на n-граммы – устойчивые (частотные) сочетания по N слов:

- nltk.bigrams() – сочетания по два слова
- nltk.trigrams() – сочетания по три слова
- nltk.ngrams(list, n) – сочетания по N слов

In [ ]:
bigramms = nltk.bigrams(clean_tokens(text))
print(Counter(bigramms).most_common(20))

trigramms = nltk.trigrams(clean_tokens(text))
print(Counter(trigramms).most_common(10))

ngramms = nltk.ngrams(clean_tokens(text), 4)
print(Counter(ngramms).most_common(5))

### Частотный анализ
Многие компьтерные методы анализа текста основаны на статистике — в нашем случае это частотность символов / словоформ / лексем / биграмм / триграмм / частей речи и т.д., ее отношение к длине текста, средняя длина текстов и т.д.

**Закон Ципфа**

Закон Ципфа («ранг—частота») — эмпирическая закономерность распределения частоты слов естественного языка: если все слова языка (или просто достаточно длинного текста) упорядочить по убыванию частоты их использования, то частота n-го слова в таком списке окажется приблизительно обратно пропорциональной его порядковому номеру n (т.н. рангу этого слова). Например, второе по используемости слово встречается примерно в два раза реже, чем первое, третье — в три раза реже, чем первое, и т.д.

Если закон Ципфа соблюдается — значит, перед нами нормальный текст на естественном языке. Если нет, то что-то с ним не так.

**Закон Хипса**

Закон Хипса — эмпирическая закономерность в лингвистике, описывающая распределение числа уникальных слов в документе (или наборе документов) как функцию от его длины.

Чем больше коллекция текстов, тем меньше новых токенов появляется с её пополнением

### Способы считать частоту
**Абсолютная частота слова**

Количество употреблений слова в тексте. Она не всегда уместна.

**Относительная частота слова**

это отношение его абсолютной частоты к какой-нибудь другой величине, например, к длине текста или корпуса. Существуют разные способы подсчета относительной частоты.

**IPM**

Для сравнения частот в разных коллекциях текстов популярен ipm (items per million) - отношение абсолютной частоты какого-либо элемента к объему корпуса, умноженное на миллион.

Метрика IPM позволяет сравнивать тексты через их характеристики. Например, "Я" заметно чаще встречается в корпусе любительской литературы, чем в корпусе художественных произведений из НКРЯ.

Кстати, есть библиотеки с подсчетом ipm

- [разные языки](https://pypi.org/project/wordfreq/)
- отдельно [русский язык](https://pypi.org/project/ruword-frequency/)

In [ ]:
# можем быстро попробовать
!pip install ruword-frequency
from ruword_frequency import Frequency

In [ ]:
# как часто слово "топор" встречается в корпусе текстов на русском?
freq = Frequency()
freq.load()

ipm = freq.ipm('топор')
print(ipm)

In [ ]:
# а как часто - в нашем тексте?
print(clean_tokens(text).count("топор") / len(clean_tokens(text)) * 1000000)

### TF-IDF
Tf-Idf - способ высоко оценить слова, которые одновременно

- показательны в документе
- не вездесущи в корпусе документов

Наивная идея такая: давайте оценка слова будет

- увеличиваться, если оно частотно в документе
- уменьшаться, если оно встречается во многих документах

В таком противостоянии победят те слова, которые выделяют документы из многих им подобных.

[Гайд](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) из библиотеки sklearn

In [ ]:
len(re.split(r'часть вторая|часть третья|часть четвертая|часть пятая|часть шестая|эпилог\b', text))

In [ ]:
corpus = re.split(r'часть вторая|часть третья|часть четвертая|часть пятая|часть шестая|эпилог\b', text)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)

In [ ]:
tfidf = tfidf_vectorizer.fit_transform(corpus)
tfidf.shape
# предположите, что означают эти числа?

In [ ]:
words = tfidf_vectorizer.get_feature_names_out()

import pandas as pd # знакомимся с pandas в следующий раз
data = tfidf.todense().tolist()
crime_and_punishment = pd.DataFrame(data, columns = words)

crime_and_punishment

In [ ]:
sorted(zip(tfidf.toarray()[0], words), reverse=True)[:10]
# топ-10 ключевых слов из первой части романа

In [ ]:
# измените код из предыдущей строки, чтобы посмотреть на ключевые слова из эпилога


In [ ]:
# @title
sorted(zip(tfidf.toarray()[6], words), reverse=True)[:10]

### Задание

Используйте архив с текстами "Девяти рассказов" Селинджера, чтобы:
- с помощью модуля shutil распакуйте архив, продите по файлам модулем os, чтобы считать все тексты в список (мини-корпус)
- циклом for предобработайте каждый текст с помощью функции, сохраните списки токенов в новый список

```[
  [токен, токен2, токен3...],
  [токен, токен2, токен3...],
  ...
]```

- постройте 2 облака слов к любым текстам из девяти на ваш выбор
- сравните топ-5 биграмм в этих текстах
- сравните ключевые слова (tf-idf) этих текстов

In [ ]:
# @title
import shutil, os
shutil.unpack_archive('Salinger.zip')
corpus = []
for i in os.listdir('Salinger'):
  with open('Salinger/' + i, 'r', encoding='utf-8') as f:
    corpus.append(f.read())

In [ ]:
# @title
clean_corpus = []
for i in corpus:
  clean_corpus.append(clean_tokens(i))

In [ ]:
# @title
wordcloud = WordCloud(background_color='white',
                      margin = 10,
                      colormap='viridis').generate_from_frequencies(Counter(clean_corpus[0]))

                      # меняем индекс - и облако построится к другому тексту

plt.imshow(wordcloud) # Что изображаем
plt.axis("off") # Без подписей на осях
plt.show() # показать изображение

In [ ]:
# @title
bigramms = nltk.bigrams(clean_corpus[0]) # аналогично меняем индекс
Counter(bigramms).most_common(5)

In [ ]:
# @title
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf = tfidf_vectorizer.fit_transform(corpus)
words = tfidf_vectorizer.get_feature_names_out()
sorted(zip(tfidf.toarray()[0], words), reverse=True)[:10] # аналогично меняем индекс